# USA vs. Europe covid-19 data
The United States and Europe are similar in size (land mass). Additionally, the united states is scattered throughout with large cities. Consequenctly, it makes sense to compare the USA to the entirety of Europe when comparing the severity of the pandmeic. I haven't see nthis map elsewhere so I decided to make it myself. 

The code is kind of a mess because I just forked it from another notebook I made, rather than starting over and organizing everything nicely.

# USA

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()
%matplotlib inline

import plotly.offline as py
from plotly import tools
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.express as px
from IPython.display import Markdown, display
from scipy.optimize import curve_fit

pd.options.mode.chained_assignment = None

# US state code to name mapping
state_map_dict = {'AL': 'Alabama',
 'AK': 'Alaska',
 'AS': 'American Samoa',
 'AZ': 'Arizona',
 'AR': 'Arkansas',
 'CA': 'California',
 'CO': 'Colorado',
 'CT': 'Connecticut',
 'DE': 'Delaware',
 'DC': 'District of Columbia',
 'D.C.': 'District of Columbia',
 'FM': 'Federated States of Micronesia',
 'FL': 'Florida',
 'GA': 'Georgia',
 'GU': 'Guam',
 'HI': 'Hawaii',
 'ID': 'Idaho',
 'IL': 'Illinois',
 'IN': 'Indiana',
 'IA': 'Iowa',
 'KS': 'Kansas',
 'KY': 'Kentucky',
 'LA': 'Louisiana',
 'ME': 'Maine',
 'MH': 'Marshall Islands',
 'MD': 'Maryland',
 'MA': 'Massachusetts',
 'MI': 'Michigan',
 'MN': 'Minnesota',
 'MS': 'Mississippi',
 'MO': 'Missouri',
 'MT': 'Montana',
 'NE': 'Nebraska',
 'NV': 'Nevada',
 'NH': 'New Hampshire',
 'NJ': 'New Jersey',
 'NM': 'New Mexico',
 'NY': 'New York',
 'NC': 'North Carolina',
 'ND': 'North Dakota',
 'MP': 'Northern Mariana Islands',
 'OH': 'Ohio',
 'OK': 'Oklahoma',
 'OR': 'Oregon',
 'PW': 'Palau',
 'PA': 'Pennsylvania',
 'PR': 'Puerto Rico',
 'RI': 'Rhode Island',
 'SC': 'South Carolina',
 'SD': 'South Dakota',
 'TN': 'Tennessee',
 'TX': 'Texas',
 'UT': 'Utah',
 'VT': 'Vermont',
 'VI': 'Virgin Islands',
 'VA': 'Virginia',
 'WA': 'Washington',
 'WV': 'West Virginia',
 'WI': 'Wisconsin',
 'WY': 'Wyoming'}

state_code_dict = {v:k for k, v in state_map_dict.items()}
state_code_dict["Chicago"] = 'Illinois'

def correct_state_names(x):
    try:
        return state_map_dict[x.split(",")[-1].strip()]
    except:
        return x.strip()
    
# us_covid19_daily.csv has numbers for states instead of names
def get_state_codes(x):
    try:
        return state_code_dict[x]
    except:
        return "Others"
    
#---print stats and such in markdown for daily updates that are easier to see    
def printmd(string):
    display(Markdown(string))
    
# ---------- color print large text -----------------
def printmdc(string, color=None):
    colorstr = "## <span style='color:{}'>{}</span>".format(color, string)
    display(Markdown(colorstr))

# ----for curve fitting-------    
def func(x, a, b, c, d):
#     return a * np.exp(b * x) + c
#     return a*b**(c*x) + d
    return a*x + b*x**2 + c*x**3 + d

# read the global data set
covid_19_df = pd.read_csv("../input/novel-corona-virus-2019-dataset/covid_19_data.csv")
# Extract only the US cases
us_covid_df = covid_19_df[covid_19_df["Country/Region"]=="US"]
# Correct state names and apply state code
us_covid_df["Province/State"] = us_covid_df["Province/State"].apply(correct_state_names)
us_covid_df["StateCode"] = us_covid_df["Province/State"].apply(lambda x: get_state_codes(x))


us_cumulative_df = us_covid_df.groupby("ObservationDate")["Confirmed", "Deaths", "Recovered"].sum().reset_index()
cumulative_df = us_cumulative_df

#fit the cumulative covid cases data to an exponential
xdata = cumulative_df["Confirmed"].index.values 
ydata = cumulative_df["Confirmed"].values
popt, pcov = curve_fit(func, xdata, ydata, p0=(0, 0, 0,0)) 
# stdv_error = np.sqrt(np.diag(pcov))
# print("std of fit error: " + str(stdv_error))
print("Covariance Matrix (squared errors are diagnols): " + str(popt))

### Plot for number of cumulative covid cases over time
fig = px.bar(cumulative_df, x="ObservationDate", y="Confirmed",color = "Deaths")
fig.add_trace(px.line(x=cumulative_df.ObservationDate, y=func(xdata, *popt)).data[0]) #comment out this line to remove fit
layout = go.Layout(
    title=go.layout.Title(
        text="Daily cumulative count of confirmed COVID-19 cases in US",
        x=0.5
    ),
    font=dict(size=14),
    width=750,
    height=450,
    xaxis_title = "Date of observation",
    yaxis_title = "Number of confirmed cases"
)
fig.update_layout(layout)
fig.show()

### Plot for number of cumulative deaths over time
# fig = px.bar(cumulative_df, x="ObservationDate", y="Deaths")
# layout = go.Layout(
#     title=go.layout.Title(
#         text="Daily cumulative count of deaths due to COVID-19 in US",
#         x=0.5
#     ),
#     font=dict(size=14),
#     width=750,
#     height=450,
#     xaxis_title = "Date of observation",
#     yaxis_title = "Number of death cases"
# )
# fig.update_layout(layout)
# fig.show()

### Plot for number of confirmed new cases over time
# cumulative_df["ConfirmedNew"] = cumulative_df["Confirmed"].diff() 
# fig = px.bar(cumulative_df, x="ObservationDate", y="ConfirmedNew")
# layout = go.Layout(
#     title=go.layout.Title(
#         text="Daily count of new confirmed COVID-19 cases in US",
#         x=0.5
#     ),
#     font=dict(size=14),
#     width=750,
#     height=450,
#     xaxis_title = "Date of observation",
#     yaxis_title = "Number of confirmed cases"
# )

# fig.update_layout(layout)
# fig.show()
print("Exponential fit parameters (a,b,c,d): " + str(popt))
# printmd("## Number of confirmed cases in US: " + str(int(cumulative_df.Confirmed.iloc[-1])))
print("Number of conrimed cases in US: " + str(str(int(cumulative_df.Confirmed.iloc[-1]))))
tomorrows_day = len(cumulative_df.index)
pred_tmrw = func(tomorrows_day, *popt)
print("(what the fit predicts for today: " + str(func(tomorrows_day-1, *popt)) +  ")")
# printmdc("Number of deaths in US: " + str(int(cumulative_df.Deaths.iloc[-1])),color='red')
print("Number of deaths in US: " + str(int(cumulative_df.Deaths.iloc[-1])))
# printmdc("Predicted confirmed cases for tomorrow: " + str(int(pred_tmrw)))
print("Predicted confirmed cases for tomorrow: " + str(int(pred_tmrw)))

In [ ]:
covid_19_df.columns = covid_19_df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '').str.replace('/','_')
covid_19_df.columns
covid_19_df.sort_values('country_region').country_region.unique()

europe_covid_df = covid_19_df[(covid_19_df.country_region =="Austria") | 
                              (covid_19_df.country_region == "Albania") |
                              (covid_19_df.country_region == "Armenia") |
                              (covid_19_df.country_region == "Belgium") |
                              (covid_19_df.country_region == "Bulgaria") |
                              (covid_19_df.country_region == "Croatia") |
                              (covid_19_df.country_region == "Czech Republic") |
                              (covid_19_df.country_region == "Denmark") | 
                              (covid_19_df.country_region == "Estonia") |
                              (covid_19_df.country_region == "Finland") |
                              (covid_19_df.country_region == "France") |
                              (covid_19_df.country_region == "Germany") |
                              (covid_19_df.country_region == "Greece") |
                              (covid_19_df.country_region == "Hungary") |
                              (covid_19_df.country_region == "Iceland") |
                              (covid_19_df.country_region == "Ireland") |
                              (covid_19_df.country_region == "Italy") |
                              (covid_19_df.country_region == "Latvia") |
                              (covid_19_df.country_region == "Lithuania") |
                              (covid_19_df.country_region == "Netherlands") |
                              (covid_19_df.country_region == "Poland") |
                              (covid_19_df.country_region == "Portugal") |
                              (covid_19_df.country_region == "Romania") |
                              (covid_19_df.country_region == "Slovakia") |
                              (covid_19_df.country_region == "Slovenia") |
                              (covid_19_df.country_region == "Spain") |
                              (covid_19_df.country_region == "Sweden") |
                              (covid_19_df.country_region == "UK") |
                              (covid_19_df.country_region == "Russia") |
                              (covid_19_df.country_region == "Switzerland") |
                              (covid_19_df.country_region == "Ukraine") |
                              (covid_19_df.country_region == "Belarus") |
                              (covid_19_df.country_region == "Norway") |
                              (covid_19_df.country_region == "San Marino") |
                              (covid_19_df.country_region == "Malta") |
                              (covid_19_df.country_region == "Moldova") |
                              (covid_19_df.country_region == "Liechtenstein") |
                              (covid_19_df.country_region == "Luxembourg") 
                             ]
europe_covid_df.info()


# Europe

In [ ]:
eu_cumulative_df = europe_covid_df.groupby("observationdate")["confirmed", "deaths", "recovered"].sum().reset_index()
cumulative_df = eu_cumulative_df

#fit the cumulative covid cases data to an exponential
xdata = cumulative_df["confirmed"].index.values 
ydata = cumulative_df["confirmed"].values
popt, pcov = curve_fit(func, xdata, ydata, p0=(0, 0, 0,0)) 
# stdv_error = np.sqrt(np.diag(pcov))
# print("std of fit error: " + str(stdv_error))
print("Covariance Matrix (squared errors are diagnols): " + str(popt))

### Plot for number of cumulative covid cases over time
fig = px.bar(cumulative_df, x="observationdate", y="confirmed",color = "deaths")
fig.add_trace(px.line(x=cumulative_df.observationdate, y=func(xdata, *popt)).data[0]) #comment out this line to remove fit
layout = go.Layout(
    title=go.layout.Title(
        text="Daily cumulative count of confirmed COVID-19 cases in Europe",
        x=0.5
    ),
    font=dict(size=14),
    width=750,
    height=450,
    xaxis_title = "Date of observation",
    yaxis_title = "Number of confirmed cases"
)
fig.update_layout(layout)
fig.show()
print("Exponential fit parameters (a,b,c,d) for a*b^(c*x) + d: " + str(popt))
# printmd("## Number of confirmed cases in US: " + str(int(cumulative_df.Confirmed.iloc[-1])))
print("Number of conrimed cases in Europe: " + str(str(int(cumulative_df.confirmed.iloc[-1]))))
tomorrows_day = len(cumulative_df.index)
pred_tmrw = func(tomorrows_day, *popt)
print("(what the fit predicts for today: " + str(func(tomorrows_day-1, *popt)) +  ")")
# printmdc("Number of deaths in US: " + str(int(cumulative_df.Deaths.iloc[-1])),color='red')
print("Number of deaths in Europe: " + str(int(cumulative_df.deaths.iloc[-1])))
# printmdc("Predicted confirmed cases for tomorrow: " + str(int(pred_tmrw)))
print("Predicted confirmed cases for tomorrow: " + str(int(pred_tmrw)))

## Plot for number of cumulative deaths over time
fig = px.bar(cumulative_df, x="observationdate", y="deaths")
layout = go.Layout(
    title=go.layout.Title(
        text="Daily cumulative count of deaths due to COVID-19 in Europe",
        x=0.5
    ),
    font=dict(size=14),
    width=750,
    height=450,
    xaxis_title = "Date of observation",
    yaxis_title = "Number of death cases"
)
fig.update_layout(layout)
fig.show()

The death rate in Italy so closely matches the cumulative cases (and is ~ 10%) that I thought I was mis-printing the figures at first.

# Overlap

In [ ]:
us_cumulative_df.columns = us_cumulative_df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '').str.replace('/','_')

# easiest way to plot two lines in Plotly was to combime the dataframes and plot based on color.
eu_cumulative_df['group'] = 'Europe'
us_cumulative_df['group'] = 'USA'
combined_df = [us_cumulative_df,eu_cumulative_df]
combined_df = pd.concat(combined_df)


fig = px.line(combined_df, x="observationdate", y="confirmed", color='group')
layout = go.Layout(
    title=go.layout.Title(
        text="Daily cumulative count of confirmed COVID-19 cases",
        x=0.5
    ),
    font=dict(size=14),
    xaxis_title = "Date of observation",
    yaxis_title = "Number of confirmed cases",

    xaxis=dict(
        rangeslider=dict(
            visible = True
        )
#         ,type='date',
    )
)
fig.update_layout(layout)
fig.show()

# ply.iplot(fig)

fig = px.line(combined_df, x="observationdate", y="deaths", color='group')
layout = go.Layout(
    title=go.layout.Title(
        text="Daily cumulative count of Deaths",
        x=0.5
    ),
    font=dict(size=14),
    xaxis_title = "Date of observation",
    yaxis_title = "Number of confirmed cases",

    xaxis=dict(
        rangeslider=dict(
            visible = True
        )
#         ,type='date',
    )
)
fig.update_layout(layout)
fig.show()

In [ ]:
# us_cumulative_df.observationdate + pd.DateOffset(1)

pd.DatetimeIndex(us_cumulative_df.observationdate) + pd.DateOffset(2)

## Shifting Time
The virus spread started earlier in Europe. We can shift the time to overlap the two plots to see how they match. If they overlap really well, we could use the EU data as a forecast. 

## Currently super buggy

In [ ]:
us_cumulative_df
eu_cumulative_df['group'] = 'Europe'
us_cumulative_df['group'] = 'USA'
combined_df = [us_cumulative_df,eu_cumulative_df]
combined_df = pd.concat(combined_df)

step = 0
fig = px.line(eu_cumulative_df,x=pd.DatetimeIndex(eu_cumulative_df.observationdate), y="confirmed", color='group')
# fig = px.line(us_cumulative_df, x=pd.DatetimeIndex(us_cumulative_df.observationdate) + pd.DateOffset(step), y="confirmed", color='group')
# fig.show()
# # Add traces, one for each slider step
for step in np.arange(0, 20, 1):
    fig.add_trace(
       go.Scatter(
          visible = False,
          line = dict(color = "blue", width = 2),
          name = "US: delta = " + str(step),
          x=pd.DatetimeIndex(us_cumulative_df.observationdate) - pd.DateOffset(step),
          y=us_cumulative_df.confirmed
       )
    )
# fig.data[10].visible=True
# fig.add_trace(
#        go.Scatter(
#           visible = True,
#           line = dict(color = "red", width = 2),
#           name = "EU",
#           x=pd.DatetimeIndex(eu_cumulative_df.observationdate),
#           y=eu_cumulative_df.confirmed
#        )
# )


# Create and add slider
steps = []
for i in range(len(fig.data)):
    step = dict(
       method = "restyle",
       args = ["visible", [False] * len(fig.data)],
    )
    step["args"][1][i] = True # Toggle i'th trace to "visible"
    steps.append(step)
    sliders = [dict(active = 40, steps = steps)]
    fig.layout.update(sliders=sliders)
    
    fig.add_trace(
       go.Scatter(
          visible = True,
          line = dict(color = "red", width = 2),
          name = "EU",
          x=pd.DatetimeIndex(eu_cumulative_df.observationdate),
          y=eu_cumulative_df.confirmed
       )
)
    
fig.show()